In [5]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', '..', '..', '..'))
print('added path:', module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from model_xray.zenml.pipelines.data_creation.dataset_compilation import retreive_datasets

def img_flatten(arr):
    return arr.reshape(arr.shape[0], -1)

added path: /home/danielg/danigil/ModelXRay


In [1]:
x_train = 20
trainset_name = f'mobilenets_train_xs0{x_train}_imsize256'

ret = retreive_datasets(
    dataset_names=[trainset_name, 'famous_le_100m_test_benigns_imsize256'] +  [f'famous_le_100m_test_xs{i}_imsize256' for i in range(8, 24)]
)

X_train, y_train = ret[trainset_name]

X_test_benigns, y_test_benigns = ret['famous_le_100m_test_benigns_imsize256']
X_test_xs23, y_test_xs23 = ret['famous_le_100m_test_xs23_imsize256']

added path: /home/danielg/danigil/ModelXRay


PyTorch version 2.4.0+cu118 available.
TensorFlow version 2.14.0 available.


In [2]:
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)

X_train: (8, 100, 100)
y_train: (8,)


In [2]:
X_train_reshaped = img_flatten(X_train)
X_test_benigns_reshaped = img_flatten(X_test_benigns)
# X_test_xs23_reshaped = img_flatten(X_test_xs23)

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

clf = KNeighborsClassifier(n_neighbors=1, weights='distance',)
clf.fit(X_train_reshaped, y_train)

train_benign_idxs = np.where(y_train == 0)[0]

X_train_benign = X_train_reshaped[train_benign_idxs]
y_train_benign = y_train[train_benign_idxs]


print("benign train:", clf.score(X_train_benign, y_train_benign))
print("train:", clf.score(X_train_reshaped, y_train))

print("benign test:", clf.score(X_test_benigns_reshaped, y_test_benigns))
for i in range(8, 24):
    X_test, y_test = ret[f'famous_le_100m_test_xs{i}_imsize256']
    X_test_reshaped = img_flatten(X_test)
    print(f"xs{i} test:", clf.score(X_test_reshaped, y_test))
# print("xs23 test:", rf.score(X_test_xs23_reshaped, y_test_xs23))

benign train: 1.0
train: 1.0
benign test: 0.25
xs8 test: 0.625
xs9 test: 0.5
xs10 test: 0.625
xs11 test: 0.5
xs12 test: 0.4375
xs13 test: 0.5
xs14 test: 0.5625
xs15 test: 0.625
xs16 test: 0.625
xs17 test: 0.8125
xs18 test: 0.75
xs19 test: 0.8125
xs20 test: 0.75
xs21 test: 0.875
xs22 test: 1.0
xs23 test: 1.0


# All CNNs, All Attacks

In [6]:
dataset_name = f"allcnns_allattacks_train_imsize256"

ret = retreive_datasets(
    dataset_names=[dataset_name]
)

X,y = ret[dataset_name]

In [8]:
print(np.unique(y, return_counts=True))
X.shape

(array([0, 1]), array([ 24, 552]))


(576, 256, 256)

In [14]:
X = img_flatten(X)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42, stratify=y)

In [29]:
X_train = X[y == 0]
y_train = y[y == 0]
X_test = X[y == 1]
y_test = y[y == 1]

In [30]:
X_train.shape

(24, 65536)

In [31]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [45]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.neighbors import KNeighborsClassifier, LocalOutlierFactor

from sklearn.metrics import classification_report
from sklearn.svm import OneClassSVM

# clf = KNeighborsClassifier(n_neighbors=11, weights='distance',)
# clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

# X_train_benign = X_train[y_train == 0]

# clf = IsolationForest(max_samples=100)
# clf = OneClassSVM()
clf = LocalOutlierFactor(n_neighbors=7, novelty=True,)
clf.fit(X_train,)

def map_outliers(y):
    return [0 if i == 1 else 1 for i in y]

y_pred_train = map_outliers(clf.predict(X_train))

print("train: ")
print(classification_report(y_train, y_pred_train))

train: 
              precision    recall  f1-score   support

           0       1.00      0.92      0.96        24
           1       0.00      0.00      0.00         0

    accuracy                           0.92        24
   macro avg       0.50      0.46      0.48        24
weighted avg       1.00      0.92      0.96        24



/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [46]:
y_pred_test = map_outliers(clf.predict(X_test))

print("test: ")
print(classification_report(y_test, y_pred_test))

test: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.08      0.15       552

    accuracy                           0.08       552
   macro avg       0.50      0.04      0.07       552
weighted avg       1.00      0.08      0.15       552



/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/danielg/micromamba/envs/zenml/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)